## Sagemaker

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, layers, optimizers
from keras.layers import Dropout

import os
import gc
from timeit import default_timer as timer

Using TensorFlow backend.


In [4]:
# train_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/training/'
# val_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/validation/'
# test_dir = '/Users/arencarpenter/Desktop/Detecting_ALL_with_CNN/Data/testing/'

In [5]:
# train_img = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(train_dir,
#                                                                   target_size = (450,450), 
#                                                                   color_mode='rgb',
#                                                                   class_mode='binary')
# val_img = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(val_dir,
#                                                                 target_size = (450,450), 
#                                                                 color_mode='rgb',
#                                                                 class_mode='binary')
# test_img = image.ImageDataGenerator(rescale = 1/255).flow_from_directory(test_dir,
#                                                                 target_size = (450,450), 
#                                                                 color_mode='rgb',
#                                                                 class_mode='binary')

In [4]:
# # plot performance
# def plot_performance(hist):
#     hist_ = hist.history
#     epochs = hist.epoch

#     plt.plot(epochs, hist_['accuracy'], label='Training Accuracy')
#     plt.plot(epochs, hist_['val_accuracy'], label='Validation Accuracy')
#     plt.title('Training and validation accuracy')
#     plt.legend()
    
#     plt.figure()
#     plt.plot(epochs, hist_['loss'], label='Training loss')
#     plt.plot(epochs, hist_['val_loss'], label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()
    
#     plt.show()

In [6]:
# scores = {}

In [7]:
import os
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlow
from tensorflow.python.keras.preprocessing.image import load_img

No handlers could be found for logger "sagemaker"


In [8]:
sagemaker_session = sagemaker.Session()

In [11]:
role = sagemaker.get_execution_role()

In [9]:
bucket = "sagemaker-all-cnn"
key = "/Data/training/"
key_output = "output"                   # Path from the bucket's root to the dataset
train_instance_type='ml.t2.medium'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.t2.medium'     # The type of EC2 instance which will be used for deployment
hyperparameters={
    "learning_rate": 1e-4,
    "decay": 1e-6
}

In [10]:
train_input_path = "s3://{}/{}/training/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

In [14]:
estimator = TensorFlow(
  entry_point=os.path.join(os.path.dirname('__file__'), "start_script.py"),             # Your entry script
  role=role,
  framework_version="1.12.0",               # TensorFlow's version
  hyperparameters=hyperparameters,
  training_steps=1000,
  evaluation_steps=100,
  train_instance_count=1,                   # "The number of GPUs instances to use"
  train_instance_type=train_instance_type,
)

In [15]:
print("Training ...")
estimator.fit({'training': train_input_path, 'eval': validation_input_path})

Training ...


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: 1 validation error detected: Value 'ml.t2.medium' at 'resourceConfig.instanceType' failed to satisfy constraint: Member must satisfy enum value set: [ml.p2.xlarge, ml.m5.4xlarge, ml.m4.16xlarge, ml.c5n.xlarge, ml.p3.16xlarge, ml.m5.large, ml.p2.16xlarge, ml.c4.2xlarge, ml.c5.2xlarge, ml.c4.4xlarge, ml.c5.4xlarge, ml.c5n.18xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.c4.8xlarge, ml.g4dn.2xlarge, ml.c5.9xlarge, ml.g4dn.4xlarge, ml.c5.xlarge, ml.g4dn.16xlarge, ml.c4.xlarge, ml.g4dn.8xlarge, ml.c5n.2xlarge, ml.c5n.4xlarge, ml.c5.18xlarge, ml.p3dn.24xlarge, ml.p3.2xlarge, ml.m5.xlarge, ml.m4.10xlarge, ml.c5n.9xlarge, ml.m5.12xlarge, ml.m4.xlarge, ml.m5.24xlarge, ml.m4.2xlarge, ml.p2.8xlarge, ml.m5.2xlarge, ml.p3.8xlarge, ml.m4.4xlarge]